In [2]:
import numpy as np
import logging
import os
from pymoo.core.problem import ElementwiseProblem

In [3]:
# Set up logging
os.makedirs("log", exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("log/generated_problem.log", mode='a'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("generated_problem")

class TravelItineraryProblem(ElementwiseProblem):

    def validate_inputs(self, budget, locations, transport_matrix, num_days):
        """Validate input data and print warnings/errors"""
        logging.info("Validating optimization inputs...")
        
        # Check if we have at least one hotel
        hotels = [loc for loc in locations if loc["type"] == "hotel"]
        if not hotels:
            logging.error("No hotels found in locations data!")
        else:
            logging.info(f"Found {len(hotels)} hotels in the data")
        
        # Check if we have hawkers for meals
        hawkers = [loc for loc in locations if loc["type"] == "hawker"]
        if not hawkers:
            logging.error("No hawker centers found - meal constraints cannot be satisfied!")
        else:
            logging.info(f"Found {len(hawkers)} hawker centers in the data")
        
        # Check for attractions
        attractions = [loc for loc in locations if loc["type"] == "attraction"]
        logging.info(f"Found {len(attractions)} attractions in the data")
        
        # Validate location data completeness
        for i, loc in enumerate(locations):
            missing = []
            if loc["type"] == "attraction":
                if "entrance_fee" not in loc or loc["entrance_fee"] is None:
                    missing.append("entrance_fee")
                if "satisfaction" not in loc or loc["satisfaction"] is None:
                    missing.append("satisfaction")
                if "duration" not in loc or loc["duration"] is None:
                    missing.append("duration")
            elif loc["type"] == "hawker":
                if "rating" not in loc or loc["rating"] is None:
                    missing.append("rating")
                if "duration" not in loc or loc["duration"] is None:
                    missing.append("duration")
            
            if missing:
                logging.warning(f"Location '{loc['name']}' is missing required fields: {', '.join(missing)}")
        
        # Check transport matrix completeness
        sample_routes = 0
        missing_routes = 0
        for i, src in enumerate(locations):
            for j, dest in enumerate(locations):
                if i != j:  # Skip self-routes
                    for hour in [8, 12, 16, 20]:  # Time brackets
                        key = (src["name"], dest["name"], hour)
                        if key not in transport_matrix:
                            missing_routes += 1
                            if missing_routes <= 5:  # Only log the first few missing routes
                                logging.error(f"Missing transport data: {key}")
                        else:
                            sample_routes += 1
        
        if missing_routes > 0:
            logging.error(f"Missing {missing_routes} routes in transport matrix!")
        else:
            logging.info(f"Transport matrix contains all required routes ({sample_routes} total)")
        
        # Check budget feasibility
        hotel_cost = num_days * 50  # Using your fixed HOTEL_COST of 50
        min_food_cost = num_days * 2 * 10  # Minimum 2 meals per day at 10 each
        
        min_cost = hotel_cost + min_food_cost
        if budget < min_cost:
            logging.error(f"Budget ({budget}) is too low! Minimum needed is {min_cost} for hotel and food alone")
        else:
            logging.info(f"Budget check passed: {budget} >= minimum {min_cost} for hotel and food")
    
    def __init__(self, budget, locations, transport_matrix, num_days=1):
        
        # Add validation checks before setup
        self.validate_inputs(budget, locations, transport_matrix, num_days)
        
        # constants
        self.NUM_DAYS = num_days
        self.HOTEL_COST = 50
        self.START_TIME = 9 * 60 # everyday starts from 9 AM
        self.HARD_LIMIT_END_TIME = 22 * 60 # everyday MUST return back to hotel by 10 PM

        # Define lunch and dinner time windows (in minutes since start of day at 9 AM)
        self.LUNCH_START = 11 * 60  # 11 AM (9 AM + 2 hours)
        self.LUNCH_END = 15 * 60    # 3 PM (9 AM + 6 hours)
        self.DINNER_START = 17 * 60  # 5 PM (9 AM + 8 hours)
        self.DINNER_END = 21 * 60   # 9 PM (9 AM + 12 hours)
        
        # hard limit money spent
        self.budget = budget
        # list of destinations
        self.locations = locations
        self.num_locations = len(locations)
        self.num_attractions = len([loc for loc in self.locations if loc["type"] == "attraction"])
        self.num_hawkers = len([loc for loc in self.locations if loc["type"] == "hawker"])
        self.num_hotels = 1 # always 1

        # transportation option prices and durations. The size MUST BE num_dest * num_dest, for 24 hours
        self.transport_types = ["transit", "drive"]
        self.num_transport_types = len(self.transport_types)
        self.transport_matrix = transport_matrix

        # x_ijkl = BINARY VAR, if the route goes in day i, use transport type j, from location k, to location l
        self.x_shape = self.NUM_DAYS * self.num_transport_types * self.num_locations * self.num_locations
        # u_ik  = CONTINUOUS VAR, tracking FINISH time of the person in day i, location k
        self.u_shape = self.NUM_DAYS * self.num_locations

        num_vars = self.x_shape + self.u_shape

        lower_bound = np.concatenate([np.zeros(self.x_shape), np.full(self.u_shape, self.START_TIME)])
        upper_bound = np.concatenate([np.ones(self.x_shape), np.full(self.u_shape, self.HARD_LIMIT_END_TIME)])
        
        def calculate_constraints():
            # For counting actual inequality constraints
            g_count = 0
            
            # For each attraction, must be visited at most once as source and at most once as destination
            g_count += 2 * self.num_attractions
            
            # For each hawker, must be visited at most once per day as source and destination
            g_count += 2 * self.num_hawkers * self.NUM_DAYS
            
            # For time constraints when a route is chosen
            g_count += self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1)
            g_count += self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) * (self.num_locations - 2)
            
            # For transport type constraints (can't use both transit and drive for the same route)
            g_count += self.NUM_DAYS * self.num_locations * self.num_locations
            
            # Budget constraint
            g_count += 1
            
            # Min/max total visits constraints
            g_count += 2
            
            # For equality constraints
            h_count = 0
            
            # Exactly 2 hawker visits per day (was inequality, now equality)
            h_count += self.NUM_DAYS
            
            # Exactly 1 hawker visit during lunch and 1 during dinner (were inequalities, now equalities)
            h_count += self.NUM_DAYS * 2
            
            # Hotel must be starting point each day
            h_count += self.NUM_DAYS
            
            # Flow conservation (in = out)
            h_count += self.NUM_DAYS * self.num_locations
            
            # Return to hotel constraint
            h_count += self.NUM_DAYS
            
            # If attraction is visited as source, it must be visited as destination
            h_count += self.num_attractions
            
            return g_count, h_count

        # Calculate actual constraints
        num_inequality_constraints, num_equality_constraints = calculate_constraints()
        
        super().__init__(
            n_var=num_vars,
            n_obj=3, # INEQUALITY_CONSTRAINT_LINE
            n_ieq_constr=num_inequality_constraints,
            n_eq_constr=num_equality_constraints,
            xl=lower_bound,
            xu=upper_bound,
        )

    def get_transport_hour(self, transport_time):
        # because the transport_matrix is only bracketed to 4 groups, we find the earliest it happens
        brackets = [8, 12, 16, 20]
        transport_hour = transport_time // 60

        for bracket in reversed(brackets):
            if transport_hour >= bracket:
                return bracket

        return brackets[-1] # from 8 PM to 8 AM next day
    
    def _evaluate(self, x, out, *args, **kwargs):
        # they're ensured to be integers
        x_var = x[:self.x_shape].reshape(self.NUM_DAYS, self.num_transport_types, self.num_locations, self.num_locations)
        u_var = x[self.x_shape:].reshape(self.NUM_DAYS, self.num_locations)

        # initialize constraints
        # equality constraints
        out["H"] = []
        # inequality constraints
        out["G"] = []

        total_cost = self.NUM_DAYS * self.HOTEL_COST  # Cost starts with hotel cost for number of nights
        total_travel_time = 0
        total_satisfaction = 0

        # for every attraction, must be a source at most once
        # for every attraction, must be a destination at most once
        # NOTE that this doesn't apply to hawkers
        for k in range(self.num_locations):
            if self.locations[k]["type"] == "attraction":
                out["G"].append(np.sum(x_var[:, :, k, :]) - 1)
                out["G"].append(np.sum(x_var[:, :, :, k]) - 1)
                
                # If attraction is a source, it must also be a destination
                out["H"].append(np.sum(x_var[:, :, k, :]) - np.sum(x_var[:, :, :, k]))

        for i in range(self.NUM_DAYS):
            # u_var[i, 0] must be the smallest of u_var[i]
            out["H"].append(np.min(u_var[i, :]) - u_var[i, 0])

            for j in range(self.num_transport_types):
                for k in range(self.num_locations):

                    for l in range(1, self.num_locations): # NOTE here that hotel (index 0) isn't included in destination. It will be computed later
                        if k == l: continue
                        # every day, if the route is chosen, the time of finishing in this place is this

                        # if chosen, then time_should_finish_l must be time_finish_l, otherwise, don't matter (just put 0)
                        # you should finish attraction l at:
                        #   - time to finish k + 
                        #   - time to transport from k to l, using transport method j +
                        #   - time to play at l
                        time_finish_l = u_var[i, l]
                        transport_hour = self.get_transport_hour(u_var[i, k])
                        transport_value = self.transport_matrix[(self.locations[k]["name"], self.locations[l]["name"], transport_hour)][self.transport_types[j]]
                        time_should_finish_l = u_var[i, k] + transport_value["duration"] + self.locations[l]["duration"]
                        # if x_var[i, j, k, l] is not chosen, then this constraint don't matter
                        out["G"].append(x_var[i, j, k, l] * time_finish_l - time_should_finish_l)
                        
                        # append to total travel time spent
                        if x_var[i, j, k, l] == 1:
                            # calculate the travel
                            total_travel_time += transport_value["duration"]
                            total_cost += transport_value["price"]

                            # calculate cost and satisfaction, based on what they come to
                            # Note: not counting anything for hotel.
                            if self.locations[l]["type"] == "attraction":
                                total_cost += self.locations[l]["entrance_fee"]
                                total_satisfaction += self.locations[l]["satisfaction"]
                            elif self.locations[l]["type"] == "hawker":
                                total_cost += 10 # ASSUME eating in a hawker is ALWAYS $10
                                total_satisfaction += self.locations[l]["rating"]

            # from last place, return to hotel.
            last_place = np.argmax(u_var[i, :])
            # THIS SHOULD NEVER HAPPEN: if last_place is already the hotel, then get the second last
            if last_place == 0: # doesn't make sense, because the first constraint is u[i, 0] must be the smallest (it's starting point)
                last_place = np.argsort(u_var[i, :])[-2]
                latest_hour = self.get_transport_hour(u_var[i, last_place])
                out["H"].append(np.sum(x_var[i, :, last_place, 0]) - 1)
            else:
                latest_hour = self.get_transport_hour(u_var[i, last_place])
                # MUST go back to hotel at the end of the day
                out["H"].append(np.sum(x_var[i, :, last_place, 0]) - 1)
                # go back to hotel
                # choose whether to use which transport type
                for j in range(self.num_transport_types):
                    if x_var[i, j, last_place, 0]:
                        # pull from google maps to get the transport details if the decision is here.
                        gohome_value = self.transport_matrix[(self.locations[last_place]["name"], self.locations[0]["name"], latest_hour)][self.transport_types[j]]
                        total_travel_time += gohome_value["duration"]
                        total_cost += gohome_value["price"]
                        break

        for i in range(self.NUM_DAYS):
            
            lunch_hawker_visit = 0
            dinner_hawker_visit = 0
            
            hawker_sum = 0
            for k in range(self.num_locations):
                # every day, for every location, if it's selected as source, it must be selected as destination
                out["H"].append(np.sum(x_var[i, :, :, k]) - np.sum(x_var[i, :, k, :]))

                if self.locations[k]["type"] == "hawker":
                    hawker_sum += np.sum(x_var[i, :, k, :])
                    
                    out["G"].append(np.sum(x_var[i, :, k, :]) - 1)  # At most once as source per day
                    out["G"].append(np.sum(x_var[i, :, :, k]) - 1)  # At most once as destination per day
                    
                    # For each route ending at this hawker, check if it's during lunch time or dinner time
                    for src in range(self.num_locations):
                        if src == k:
                            continue
                        for j_transport in range(self.num_transport_types):
                            if u_var[i, k] >= self.LUNCH_START and u_var[i, k] <= self.LUNCH_END:
                                lunch_hawker_visit += x_var[i, j_transport, src, k]
                            
                            if u_var[i, k] >= self.DINNER_START and u_var[i, k] <= self.DINNER_END:
                                dinner_hawker_visit += x_var[i, j_transport, src, k]

            # every day, must go to hawkers exactly twice (lunch & dinner. Can go more times if they want to)
            out["H"].append(hawker_sum - 2)
            
            # every day, must visit a hawker during lunch time (at least one hawker visit with arrival/stay during lunch hours)
            if self.num_hawkers > 0:  # Only add constraint if there are hawkers available
                out["H"].append(lunch_hawker_visit - 1)
                
                # every day, must visit a hawker during dinner time (at least one hawker visit with arrival/stay during dinner hours)
                out["H"].append(dinner_hawker_visit - 1)

        for i in range(self.NUM_DAYS):
            for k in range(self.num_locations):
                for l in range(self.num_locations):
                    # for every day, if public transportation is chosen, taxi can't be chosen, and vice versa
                    # (sum j in transport_types x_ijkl <= 1) for all days, for all sources, for all destinations
                    out["G"].append(np.sum(x_var[i, :, k, l]) - 1)

        # finally, make sure everything is within budget
        out["G"].append(total_cost - self.budget)
        
        # Calculate reasonable minimum and maximum visits
        # Minimum: At least 2 hawker visits per day (lunch & dinner)
        min_visits = self.NUM_DAYS * 2
        # Maximum: Reasonable upper bound for visits per day (e.g., start at hotel + 2 meals + 2-3 attractions)
        max_visits_per_day = 6
        max_visits = self.NUM_DAYS * max_visits_per_day

        # Constraint: Total visits should be at least the minimum
        out["G"].append(min_visits - np.sum(x_var))
        # Constraint: Total visits should be at most the maximum
        out["G"].append(np.sum(x_var) - max_visits)
        # INDENTATION_COUNT_LINE
        # <ADD ADDITIONAL CONSTRAINTS HERE>

        # objectives
        out["F"] = [total_cost, total_travel_time, -total_satisfaction]

In [4]:
# Set up logging
os.makedirs("log", exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("log/debug_problem.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("debug_problem")

In [ ]:
def create_simplified_problem():
    """
    Create a simplified version of the travel itinerary problem with 
    just 1 day, 1 hotel, 2 hawker centers, and 3 attractions.
    """
    logger.info("Creating simplified test problem")
    
    # 1. Create simplified locations
    locations = [
        {
            "type": "hotel",
            "name": "Test Hotel",
            "lat": 1.290000,
            "lng": 103.850000,
            "duration": 0  # No time spent at hotel
        },
        {
            "type": "hawker",
            "name": "Hawker Center 1",
            "lat": 1.291000,
            "lng": 103.851000,
            "rating": 4.5,
            "duration": 60  # 60 minutes for meals
        },
        {
            "type": "hawker",
            "name": "Hawker Center 2",
            "lat": 1.292000,
            "lng": 103.852000,
            "rating": 4.0,
            "duration": 60  # 60 minutes for meals
        },
        {
            "type": "attraction",
            "name": "Attraction 1",
            "lat": 1.293000,
            "lng": 103.853000,
            "satisfaction": 8.0,
            "entrance_fee": 20,
            "duration": 60  # 60 minutes to visit
        },
        {
            "type": "attraction",
            "name": "Attraction 2",
            "lat": 1.294000,
            "lng": 103.854000,
            "satisfaction": 7.5,
            "entrance_fee": 15,
            "duration": 60  # 60 minutes to visit
        },
        {
            "type": "attraction",
            "name": "Attraction 3",
            "lat": 1.295000,
            "lng": 103.855000,
            "satisfaction": 9.0,
            "entrance_fee": 25,
            "duration": 60  # 60 minutes to visit
        }
    ]
    
    # 2. Create a simplified transport matrix
    transport_matrix = {}
    
    # Define time brackets
    time_brackets = [8, 12, 16, 20]
    
    # Generate routes for all location pairs in all time brackets
    for hour in time_brackets:
        for i, src in enumerate(locations):
            for j, dest in enumerate(locations):
                if i != j:  # Skip self-routes
                    # Create key for this route
                    key = (src["name"], dest["name"], hour)
                    
                    # Generate reasonable travel times and costs
                    # Simple formula: ~10 minutes per km and ~$1 per km
                    # Distance roughly approximated based on lat/lng
                    distance_km = np.sqrt((src["lat"] - dest["lat"])**2 + (src["lng"] - dest["lng"])**2) * 100
                    
                    transport_matrix[key] = {
                        "transit": {
                            "duration": max(10, round(distance_km * 10)),  # Min 10 minutes
                            "price": round(distance_km * 0.5, 2),  # Transit cheaper
                        },
                        "drive": {
                            "duration": max(5, round(distance_km * 8)),  # Driving faster
                            "price": round(distance_km * 1.0, 2),  # Driving more expensive
                        }
                    }
    
    # 3. Set a reasonable budget
    budget = 150  # This should be enough for 1 day
    
    # 4. Create the problem
    problem = TravelItineraryProblem(
        budget=budget,
        locations=locations,
        transport_matrix=transport_matrix,
        num_days=1  # Just 1 day
    )
    
    return problem

# Add the debugging methods to TravelItineraryProblem
def add_debug_methods_to_problem():
    """
    Add the debugging methods to the TravelItineraryProblem class.
    This function modifies the class directly.
    """
    # Debug constraint violations
    TravelItineraryProblem.debug_constraint_violations = debug_constraint_violations
    
    # Get constraint description
    TravelItineraryProblem.get_constraint_description = get_constraint_description
    
    # Run feasibility analysis
    TravelItineraryProblem.run_feasibility_analysis = run_feasibility_analysis

# Your debugging methods (copied from previous artifact)
def debug_constraint_violations(self, x, verbose=True):
    """
    Debug constraint violations by evaluating a random or provided solution
    and reporting which constraints are violated.
    
    Args:
        x: A candidate solution to evaluate, or None to generate a random one
        verbose: Whether to print detailed information about violations
        
    Returns:
        A dictionary with statistics about constraint violations
    """
    # If no solution provided, generate a random one within bounds
    if x is None:
        x = np.random.random(self.n_var)
        x = np.minimum(np.maximum(x, self.xl), self.xu)
    
    # Evaluate the solution
    out = {}
    self._evaluate(x, out)
    
    # Initialize violation tracking
    violations = {
        "equality": [],
        "inequality": [],
        "worst_eq_violation": 0,
        "worst_ineq_violation": 0,
        "total_eq_violations": 0,
        "total_ineq_violations": 0
    }
    
    # Check equality constraints (H)
    if "H" in out and len(out["H"]) > 0:
        eq_violations = np.abs(out["H"])
        violations["worst_eq_violation"] = np.max(eq_violations)
        violations["total_eq_violations"] = np.sum(eq_violations > 0.001)
        
        # Track individual violations
        for i, val in enumerate(eq_violations):
            if val > 0.001:  # Tolerance threshold
                violations["equality"].append((i, val))
    
    # Check inequality constraints (G)
    if "G" in out and len(out["G"]) > 0:
        ineq_violations = np.maximum(0, out["G"])
        violations["worst_ineq_violation"] = np.max(ineq_violations)
        violations["total_ineq_violations"] = np.sum(ineq_violations > 0.001)
        
        # Track individual violations
        for i, val in enumerate(ineq_violations):
            if val > 0.001:  # Tolerance threshold
                violations["inequality"].append((i, val))
    
    # Sort violations by severity
    violations["equality"].sort(key=lambda x: x[1], reverse=True)
    violations["inequality"].sort(key=lambda x: x[1], reverse=True)
    
    # Report violations
    if verbose:
        print(f"\n===== CONSTRAINT VIOLATION ANALYSIS =====")
        print(f"Equality Constraints: {violations['total_eq_violations']} violated (out of {len(out.get('H', []))})")
        print(f"Inequality Constraints: {violations['total_ineq_violations']} violated (out of {len(out.get('G', []))})")
        
        if violations["equality"]:
            print("\nTop Equality Constraint Violations:")
            for i, (idx, val) in enumerate(violations["equality"][:10]):  # Show top 10
                constraint_desc = self.get_constraint_description("H", idx)
                print(f"  {i+1}. H[{idx}]: {val:.4f} - {constraint_desc}")
        
        if violations["inequality"]:
            print("\nTop Inequality Constraint Violations:")
            for i, (idx, val) in enumerate(violations["inequality"][:10]):  # Show top 10
                constraint_desc = self.get_constraint_description("G", idx)
                print(f"  {i+1}. G[{idx}]: {val:.4f} - {constraint_desc}")
        
        print(f"Objective values: {out.get('F', 'N/A')}")
        print("=========================================\n")
    
    return violations

def get_constraint_description(self, type, idx):
    """
    Get a human-readable description of a constraint based on its index.
    
    Args:
        type: "G" for inequality or "H" for equality constraint
        idx: The index of the constraint
        
    Returns:
        A string describing the constraint
    """
    # This mapping needs to be customized based on your problem
    if type == "G":
        # Track position in the constraint array
        pos = 0
        
        # Attraction visit limits
        if idx < 2 * self.num_attractions:
            attraction_idx = idx // 2
            attraction_name = self.locations[attraction_idx]["name"]
            if idx % 2 == 0:
                return f"Attraction '{attraction_name}' visited at most once as source"
            else:
                return f"Attraction '{attraction_name}' visited at most once as destination"
        pos += 2 * self.num_attractions
        
        # Hawker visit limits per day
        if idx < pos + 2 * self.num_hawkers * self.NUM_DAYS:
            rel_idx = idx - pos
            day_idx = rel_idx // (2 * self.num_hawkers)
            hawker_rel_idx = (rel_idx % (2 * self.num_hawkers)) // 2
            
            # Find the actual hawker index in locations
            hawker_idx = -1
            hawker_count = 0
            for i, loc in enumerate(self.locations):
                if loc["type"] == "hawker":
                    if hawker_count == hawker_rel_idx:
                        hawker_idx = i
                        break
                    hawker_count += 1
            
            hawker_name = self.locations[hawker_idx]["name"] if hawker_idx >= 0 else f"Hawker {hawker_rel_idx}"
            
            if rel_idx % 2 == 0:
                return f"Day {day_idx+1}: Hawker '{hawker_name}' visited at most once as source"
            else:
                return f"Day {day_idx+1}: Hawker '{hawker_name}' visited at most once as destination"
        pos += 2 * self.num_hawkers * self.NUM_DAYS
        
        # Time constraints (many of these, simplify description)
        time_constraints_count = self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) 
        time_constraints_count += self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) * (self.num_locations - 2)
        
        if idx < pos + time_constraints_count:
            return "Time constraint for route timing"
        pos += time_constraints_count
        
        # Transport type constraints
        if idx < pos + self.NUM_DAYS * self.num_locations * self.num_locations:
            rel_idx = idx - pos
            day = rel_idx // (self.num_locations * self.num_locations)
            remaining = rel_idx % (self.num_locations * self.num_locations)
            src = remaining // self.num_locations
            dest = remaining % self.num_locations
            return f"Day {day+1}: Max one transport type from {self.locations[src]['name']} to {self.locations[dest]['name']}"
        pos += self.NUM_DAYS * self.num_locations * self.num_locations
        
        # Budget constraint
        if idx == pos:
            return "Total cost within budget"
        pos += 1
        
        # Min/max visit constraints
        if idx == pos:
            return "Minimum required visits"
        elif idx == pos + 1:
            return "Maximum allowed visits"
        
        return f"Unknown inequality constraint {idx}"
        
    elif type == "H":
        # Track position in the constraint array
        pos = 0
        
        # Attraction source = destination constraint
        if idx < self.num_attractions:
            attraction_idx = -1
            attraction_count = 0
            for i, loc in enumerate(self.locations):
                if loc["type"] == "attraction":
                    if attraction_count == idx:
                        attraction_idx = i
                        break
                    attraction_count += 1
            
            attraction_name = self.locations[attraction_idx]["name"] if attraction_idx >= 0 else f"Attraction {idx}"
            return f"Attraction '{attraction_name}' must be both source and destination if visited"
        pos += self.num_attractions
        
        # Hotel starting point constraint
        if idx < pos + self.NUM_DAYS:
            day = idx - pos
            return f"Day {day+1}: Hotel must be starting point"
        pos += self.NUM_DAYS
        
        # Flow conservation
        if idx < pos + self.NUM_DAYS * self.num_locations:
            rel_idx = idx - pos
            day = rel_idx // self.num_locations
            loc = rel_idx % self.num_locations
            return f"Day {day+1}: Flow conservation at {self.locations[loc]['name']}"
        pos += self.NUM_DAYS * self.num_locations
        
        # Return to hotel
        if idx < pos + self.NUM_DAYS:
            day = idx - pos
            return f"Day {day+1}: Must return to hotel"
        pos += self.NUM_DAYS
        
        # Exactly 2 hawker visits per day
        if idx < pos + self.NUM_DAYS:
            day = idx - pos
            return f"Day {day+1}: Exactly 2 hawker visits"
        pos += self.NUM_DAYS
        
        # Exactly 1 lunch and 1 dinner hawker visit
        if idx < pos + self.NUM_DAYS * 2:
            rel_idx = idx - pos
            day = rel_idx // 2
            meal = "lunch" if rel_idx % 2 == 0 else "dinner"
            return f"Day {day+1}: Exactly 1 hawker visit during {meal}"
        
        return f"Unknown equality constraint {idx}"
    
    return "Unknown constraint type"

def run_feasibility_analysis(self, num_samples=10):
    """
    Run a feasibility analysis by testing multiple random solutions.
    
    Args:
        num_samples: Number of random solutions to test
        
    Returns:
        Statistics about constraint violations across all samples
    """
    print(f"\n===== RUNNING FEASIBILITY ANALYSIS WITH {num_samples} SAMPLES =====")
    
    # Track the most commonly violated constraints
    eq_violation_counts = {}
    ineq_violation_counts = {}
    
    # Track overall violation statistics
    total_eq_violations = 0
    total_ineq_violations = 0
    best_total_violations = float('inf')
    best_solution = None
    
    for i in range(num_samples):
        # Generate a random solution
        x = np.random.random(self.n_var)
        x = np.minimum(np.maximum(x, self.xl), self.xu)
        
        # Evaluate constraints (without verbose output)
        violations = self.debug_constraint_violations(x, verbose=False)
        
        # Count total violations for this sample
        total_violations = violations["total_eq_violations"] + violations["total_ineq_violations"]
        
        # Update best solution if this one has fewer violations
        if total_violations < best_total_violations:
            best_total_violations = total_violations
            best_solution = x.copy()
        
        # Update violation counts
        for idx, val in violations["equality"]:
            eq_violation_counts[idx] = eq_violation_counts.get(idx, 0) + 1
        
        for idx, val in violations["inequality"]:
            ineq_violation_counts[idx] = ineq_violation_counts.get(idx, 0) + 1
        
        total_eq_violations += violations["total_eq_violations"]
        total_ineq_violations += violations["total_ineq_violations"]
        
        # Print progress
        if (i+1) % 10 == 0:
            print(f"  Processed {i+1}/{num_samples} samples...")
    
    # Calculate average violations
    avg_eq_violations = total_eq_violations / num_samples
    avg_ineq_violations = total_ineq_violations / num_samples
    
    # Sort violation counts
    sorted_eq_violations = sorted(eq_violation_counts.items(), key=lambda x: x[1], reverse=True)
    sorted_ineq_violations = sorted(ineq_violation_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Print summary
    print("\n----- FEASIBILITY ANALYSIS SUMMARY -----")
    print(f"Average equality violations per sample: {avg_eq_violations:.2f}")
    print(f"Average inequality violations per sample: {avg_ineq_violations:.2f}")
    
    print("\nMost frequently violated equality constraints:")
    for i, (idx, count) in enumerate(sorted_eq_violations[:5]):
        percent = (count / num_samples) * 100
        constraint_desc = self.get_constraint_description("H", idx)
        print(f"  {i+1}. H[{idx}]: {percent:.1f}% of samples - {constraint_desc}")
    
    print("\nMost frequently violated inequality constraints:")
    for i, (idx, count) in enumerate(sorted_ineq_violations[:5]):
        percent = (count / num_samples) * 100
        constraint_desc = self.get_constraint_description("G", idx)
        print(f"  {i+1}. G[{idx}]: {percent:.1f}% of samples - {constraint_desc}")
    
    print("\n----- BEST SOLUTION ANALYSIS -----")
    if best_solution is not None:
        self.debug_constraint_violations(best_solution, verbose=True)
    
    print("========================================")
    
    return {
        "eq_violation_counts": eq_violation_counts,
        "ineq_violation_counts": ineq_violation_counts,
        "avg_eq_violations": avg_eq_violations,
        "avg_ineq_violations": avg_ineq_violations,
        "best_solution": best_solution
    }

def debug_simplified_problem():
    """Main function to debug the simplified problem"""
    # Add debugging methods to the problem class
    add_debug_methods_to_problem()
    
    # Create simplified problem
    problem = create_simplified_problem()
    
    # Print problem characteristics
    print("\n===== SIMPLIFIED PROBLEM CHARACTERISTICS =====")
    print(f"Number of locations: {problem.num_locations}")
    print(f"  - Hotels: {problem.num_hotels}")
    print(f"  - Hawkers: {problem.num_hawkers}")
    print(f"  - Attractions: {problem.num_attractions}")
    print(f"Number of days: {problem.NUM_DAYS}")
    print(f"Budget: ${problem.budget}")
    print(f"Transport matrix entries: {len(problem.transport_matrix)}")
    print(f"Number of variables: {problem.n_var}")
    print(f"Number of inequality constraints: {problem.n_ieq_constr}")
    print(f"Number of equality constraints: {problem.n_eq_constr}")
    
    # Run feasibility analysis
    analysis = problem.run_feasibility_analysis(num_samples=100)
    
    # Make suggestions for fixing the problem
    print("\n===== SUGGESTIONS FOR FIXING THE PROBLEM =====")
    
    # Check for common issues
    if "Exactly 2 hawker visits" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n1. Issue with 'Exactly 2 hawker visits per day':")
        print("   - Try relaxing to 'at least 1 hawker visit per day'")
        print("   - Modify the constraint in _evaluate() from:")
        print("     out['H'].append(hawker_sum - 2)  # Exactly 2")
        print("   - To:")
        print("     out['G'].append(1 - hawker_sum)  # At least 1")
    
    if "Exactly 1 hawker visit during lunch" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n2. Issue with 'Exactly 1 hawker visit during lunch/dinner':")
        print("   - Check if lunch/dinner windows are wide enough")
        print("   - Current lunch window: {}-{} minutes".format(problem.LUNCH_START, problem.LUNCH_END))
        print("   - Current dinner window: {}-{} minutes".format(problem.DINNER_START, problem.DINNER_END))
        print("   - Try widening these windows or relaxing to 'at least' constraints")
    
    if "Flow conservation" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n3. Issue with 'Flow conservation':")
        print("   - This might conflict with 'Hotel must be starting point' and 'Must return to hotel'")
        print("   - Make sure these constraints are compatible")
        print("   - Consider special handling for the hotel's flow conservation")
    
    if "Time constraint for route timing" in ' '.join(problem.get_constraint_description("G", idx) for idx in analysis["ineq_violation_counts"]):
        print("\n4. Issue with 'Time constraints for route timing':")
        print("   - Check if the transport durations allow completing all required visits")
        print("   - Available time per day: {} minutes".format(problem.HARD_LIMIT_END_TIME - problem.START_TIME))
        print("   - Try increasing the daily time limit or shortening activity durations")
    
    # Suggest adjustments based on analysis
    print("\n===== RECOMMENDED CODE ADJUSTMENTS =====")
    print("Based on the analysis, here are some code adjustments you can try:")
    
    eq_issues = sorted(analysis["eq_violation_counts"].items(), key=lambda x: x[1], reverse=True)[:3]
    ineq_issues = sorted(analysis["ineq_violation_counts"].items(), key=lambda x: x[1], reverse=True)[:3]
    
    if eq_issues:
        print("\nFor top equality constraint issues:")
        for idx, count in eq_issues:
            desc = problem.get_constraint_description("H", idx)
            print(f"  - H[{idx}] ({count} violations): {desc}")
    
    if ineq_issues:
        print("\nFor top inequality constraint issues:")
        for idx, count in ineq_issues:
            desc = problem.get_constraint_description("G", idx)
            print(f"  - G[{idx}] ({count} violations): {desc}")

In [ ]:
def visualize_solution(self, x, show_full_matrix=False):
    """
    Visualize a solution in a human-readable format to understand what routes are being chosen
    
    Args:
        x: The solution vector to visualize
        show_full_matrix: Whether to show the full decision matrix or just the selected routes
    """
    print("\n===== SOLUTION VISUALIZATION =====")
    
    # Extract decision variables from solution vector
    x_var = x[:self.x_shape].reshape(self.NUM_DAYS, self.num_transport_types, self.num_locations, self.num_locations)
    u_var = x[self.x_shape:].reshape(self.NUM_DAYS, self.num_locations)
    
    # Visualize each day
    for day in range(self.NUM_DAYS):
        print(f"\nDAY {day+1} SCHEDULE:")
        print("-" * 50)
        
        # Extract decision matrix for this day
        day_matrix = x_var[day]
        
        # Find starting point (should be hotel)
        # Also collect all chosen routes for this day
        chosen_routes = []
        
        for src in range(self.num_locations):
            for dest in range(self.num_locations):
                if src == dest:
                    continue
                
                for transport_idx in range(self.num_transport_types):
                    # Check if this route is selected
                    if day_matrix[transport_idx, src, dest] > 0.5:
                        transport_mode = self.transport_types[transport_idx]
                        chosen_routes.append((src, dest, transport_mode, day_matrix[transport_idx, src, dest]))
        
        # Sort chosen routes by source finish time
        chosen_routes.sort(key=lambda route: u_var[day, route[0]])
        
        # Display chosen routes in order
        if chosen_routes:
            # Track the current location and time
            current_location = 0  # Start at hotel (index 0)
            current_time = self.START_TIME  # Start time of the day
            
            for i, (src, dest, transport_mode, decision_val) in enumerate(chosen_routes):
                # Check if this is a valid next step (source should be current location)
                if src != current_location:
                    print(f"WARNING: Route sequence broken - current location is {self.locations[current_location]['name']} but next source is {self.locations[src]['name']}")
                
                # Get source and destination information
                src_name = self.locations[src]["name"]
                dest_name = self.locations[dest]["name"]
                src_type = self.locations[src]["type"]
                dest_type = self.locations[dest]["type"]
                
                # Get transport information
                transport_hour = self.get_transport_hour(u_var[day, src])
                transport_key = (src_name, dest_name, transport_hour)
                
                if transport_key in self.transport_matrix:
                    transport_info = self.transport_matrix[transport_key][transport_mode]
                    transport_duration = transport_info["duration"]
                    transport_price = transport_info["price"]
                else:
                    print(f"WARNING: Missing transport information for {transport_key}")
                    transport_duration = 0
                    transport_price = 0
                
                # Calculate arrival time at destination
                dest_start_time = current_time + transport_duration
                dest_finish_time = dest_start_time + self.locations[dest]["duration"]
                
                # Format times as HH:MM
                current_time_str = f"{(current_time + self.START_TIME) // 60:02d}:{(current_time + self.START_TIME) % 60:02d}"
                dest_start_time_str = f"{(dest_start_time + self.START_TIME) // 60:02d}:{(dest_start_time + self.START_TIME) % 60:02d}"
                dest_finish_time_str = f"{(dest_finish_time + self.START_TIME) // 60:02d}:{(dest_finish_time + self.START_TIME) % 60:02d}"
                
                # Check for lunch/dinner status for hawkers
                meal_status = ""
                if dest_type == "hawker":
                    if dest_start_time >= self.LUNCH_START and dest_start_time <= self.LUNCH_END:
                        meal_status = " (LUNCH)"
                    elif dest_start_time >= self.DINNER_START and dest_start_time <= self.DINNER_END:
                        meal_status = " (DINNER)"
                    else:
                        meal_status = " (OUTSIDE MEAL TIMES)"
                
                # Print route information
                print(f"{i+1}. {src_name} ({src_type}) -> {dest_name} ({dest_type}){meal_status}")
                print(f"   Leave at {current_time_str}, arrive at {dest_start_time_str}, finish at {dest_finish_time_str}")
                print(f"   Transport: {transport_mode.upper()} ({transport_duration} min, ${transport_price:.2f})")
                
                if dest_type == "attraction":
                    print(f"   Entrance fee: ${self.locations[dest]['entrance_fee']:.2f}")
                elif dest_type == "hawker":
                    print(f"   Meal cost: $10.00")
                
                # Update current location and time for next route
                current_location = dest
                current_time = dest_finish_time
            
            # Check if we returned to the hotel at the end
            if current_location != 0:
                print(f"WARNING: Day does not end at hotel. Last location: {self.locations[current_location]['name']}")
        else:
            print("No routes selected for this day.")
        
        # Display time variables for each location
        print("\nLocation Finish Times:")
        for loc in range(self.num_locations):
            loc_time = u_var[day, loc]
            if loc_time > self.START_TIME:  # Only show visited locations
                loc_time_str = f"{(loc_time + self.START_TIME) // 60:02d}:{(loc_time + self.START_TIME) % 60:02d}"
                print(f"  {self.locations[loc]['name']}: {loc_time_str}")
    
    # Show full decision matrix if requested
    if show_full_matrix:
        print("\nFull Decision Matrix:")
        for day in range(self.NUM_DAYS):
            print(f"\nDay {day+1}:")
            for transport_idx, mode in enumerate(self.transport_types):
                print(f"\nTransport Mode: {mode.upper()}")
                
                # Create a table header
                header = "       | "
                for dest in range(self.num_locations):
                    header += f"{self.locations[dest]['name'][:8]:<8} | "
                print(header)
                print("-" * len(header))
                
                # Print each row
                for src in range(self.num_locations):
                    row = f"{self.locations[src]['name'][:8]:<8} | "
                    for dest in range(self.num_locations):
                        val = x_var[day, transport_idx, src, dest]
                        if val > 0.01:
                            row += f"{val:.2f}".ljust(8) + " | "
                        else:
                            row += "        | "
                    print(row)
    
    print("\n===== END OF SOLUTION VISUALIZATION =====")

# Update the debug_constraint_violations method to include visualization
def debug_constraint_violations_with_viz(self, x, verbose=True, visualize=True):
    """
    Enhanced version of debug_constraint_violations that also visualizes the solution
    """
    # Perform the standard debug
    violations = self.debug_constraint_violations(x, verbose)
    
    # Visualize the solution if requested
    if visualize:
        self.visualize_solution(x, show_full_matrix=False)
    
    return violations

# Update the run_feasibility_analysis method to find the best solution and visualize it
def run_feasibility_analysis_with_viz(self, num_samples=10):
    """
    Enhanced version of run_feasibility_analysis that visualizes the best solution found
    """
    # Run the standard feasibility analysis
    analysis = self.run_feasibility_analysis(num_samples)
    
    # Visualize the best solution found
    if analysis["best_solution"] is not None:
        print("\n===== VISUALIZING BEST SOLUTION FOUND =====")
        self.visualize_solution(analysis["best_solution"])
    
    return analysis

# Add these methods to the updated debugging setup
def add_enhanced_debug_methods_to_problem():
    """
    Add the enhanced debugging methods including visualization to the TravelItineraryProblem class.
    """
    # Add the standard debug methods
    add_debug_methods_to_problem()
    
    # Add the visualization method
    TravelItineraryProblem.visualize_solution = visualize_solution
    
    # Add the enhanced methods
    TravelItineraryProblem.debug_constraint_violations_with_viz = debug_constraint_violations_with_viz
    TravelItineraryProblem.run_feasibility_analysis_with_viz = run_feasibility_analysis_with_viz

# Updated main function to use the enhanced methods
def debug_simplified_problem_with_viz():
    """Main function to debug the simplified problem with visualization"""
    # Add debugging methods to the problem class
    add_enhanced_debug_methods_to_problem()
    
    # Create simplified problem
    problem = create_simplified_problem()
    
    # Print problem characteristics
    print("\n===== SIMPLIFIED PROBLEM CHARACTERISTICS =====")
    print(f"Number of locations: {problem.num_locations}")
    print(f"  - Hotels: {problem.num_hotels}")
    print(f"  - Hawkers: {problem.num_hawkers}")
    print(f"  - Attractions: {problem.num_attractions}")
    print(f"Number of days: {problem.NUM_DAYS}")
    print(f"Budget: ${problem.budget}")
    print(f"Transport matrix entries: {len(problem.transport_matrix)}")
    print(f"Number of variables: {problem.n_var}")
    print(f"Number of inequality constraints: {problem.n_ieq_constr}")
    print(f"Number of equality constraints: {problem.n_eq_constr}")
    
    # Print time windows
    print("\nTime Windows:")
    print(f"  Daily start time: {problem.START_TIME//60:02d}:{problem.START_TIME%60:02d}")
    print(f"  Daily end time: {problem.HARD_LIMIT_END_TIME//60:02d}:{problem.HARD_LIMIT_END_TIME%60:02d}")
    print(f"  Lunch window: {problem.LUNCH_START//60:02d}:{problem.LUNCH_START%60:02d} - {problem.LUNCH_END//60:02d}:{problem.LUNCH_END%60:02d}")
    print(f"  Dinner window: {problem.DINNER_START//60:02d}:{problem.DINNER_START%60:02d} - {problem.DINNER_END//60:02d}:{problem.DINNER_END%60:02d}")
    
    # Run enhanced feasibility analysis with visualization
    analysis = problem.run_feasibility_analysis_with_viz(num_samples=100)
    
    # Make suggestions for fixing the problem
    print("\n===== SUGGESTIONS FOR FIXING THE PROBLEM =====")
    
    # Check for common issues
    if "Exactly 2 hawker visits" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n1. Issue with 'Exactly 2 hawker visits per day':")
        print("   - Try relaxing to 'at least 1 hawker visit per day'")
        print("   - Modify the constraint in _evaluate() from:")
        print("     out['H'].append(hawker_sum - 2)  # Exactly 2")
        print("   - To:")
        print("     out['G'].append(1 - hawker_sum)  # At least 1")
    
    if "Exactly 1 hawker visit during lunch" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n2. Issue with 'Exactly 1 hawker visit during lunch/dinner':")
        print("   - Check if lunch/dinner windows are wide enough")
        print("   - Current lunch window: {}-{} minutes".format(problem.LUNCH_START, problem.LUNCH_END))
        print("   - Current dinner window: {}-{} minutes".format(problem.DINNER_START, problem.DINNER_END))
        print("   - Try widening these windows or relaxing to 'at least' constraints")
    
    if "Flow conservation" in ' '.join(problem.get_constraint_description("H", idx) for idx in analysis["eq_violation_counts"]):
        print("\n3. Issue with 'Flow conservation':")
        print("   - This might conflict with 'Hotel must be starting point' and 'Must return to hotel'")
        print("   - Make sure these constraints are compatible")
        print("   - Consider special handling for the hotel's flow conservation")
    
    if "Time constraint for route timing" in ' '.join(problem.get_constraint_description("G", idx) for idx in analysis["ineq_violation_counts"]):
        print("\n4. Issue with 'Time constraints for route timing':")
        print("   - Check if the transport durations allow completing all required visits")
        print("   - Available time per day: {} minutes".format(problem.HARD_LIMIT_END_TIME - problem.START_TIME))
        print("   - Try increasing the daily time limit or shortening activity durations")
    
    # Check for potential time feasibility issues based on best solution visualization
    if analysis["best_solution"] is not None:
        # Parse the solution
        x_var = analysis["best_solution"][:problem.x_shape].reshape(problem.NUM_DAYS, problem.num_transport_types, problem.num_locations, problem.num_locations)
        u_var = analysis["best_solution"][problem.x_shape:].reshape(problem.NUM_DAYS, problem.num_locations)
        
        # Check for lunch/dinner scheduling
        lunch_visits = 0
        dinner_visits = 0
        hawker_visits = 0
        
        for day in range(problem.NUM_DAYS):
            for transport_idx in range(problem.num_transport_types):
                for src in range(problem.num_locations):
                    for dest in range(problem.num_locations):
                        if x_var[day, transport_idx, src, dest] > 0.5:
                            if problem.locations[dest]["type"] == "hawker":
                                hawker_visits += 1
                                arrival_time = u_var[day, dest] - problem.locations[dest]["duration"]
                                
                                if arrival_time >= problem.LUNCH_START and arrival_time <= problem.LUNCH_END:
                                    lunch_visits += 1
                                if arrival_time >= problem.DINNER_START and arrival_time <= problem.DINNER_END:
                                    dinner_visits += 1
        
        print("\n5. Time window analysis from best solution:")
        print(f"   - Total hawker visits in best solution: {hawker_visits}")
        print(f"   - Hawker visits during lunch window: {lunch_visits}")
        print(f"   - Hawker visits during dinner window: {dinner_visits}")
        
        if hawker_visits < 2 * problem.NUM_DAYS:
            print("   - The best solution doesn't have enough hawker visits. Try relaxing the constraint.")
        
        if lunch_visits < problem.NUM_DAYS or dinner_visits < problem.NUM_DAYS:
            print("   - The best solution doesn't satisfy lunch/dinner requirements. Try widening time windows.")
    
    # Suggest adjustments based on analysis
    print("\n===== RECOMMENDED CODE ADJUSTMENTS =====")
    print("Based on the visualization and analysis, here are the most likely issues:")
    
    eq_issues = sorted(analysis["eq_violation_counts"].items(), key=lambda x: x[1], reverse=True)[:3]
    ineq_issues = sorted(analysis["ineq_violation_counts"].items(), key=lambda x: x[1], reverse=True)[:3]
    
    if eq_issues:
        print("\nTop equality constraint issues:")
        for idx, count in eq_issues:
            desc = problem.get_constraint_description("H", idx)
            print(f"  - H[{idx}] ({count} violations): {desc}")
    
    if ineq_issues:
        print("\nTop inequality constraint issues:")
        for idx, count in ineq_issues:
            desc = problem.get_constraint_description("G", idx)
            print(f"  - G[{idx}] ({count} violations): {desc}")

In [10]:
debug_simplified_problem_with_viz()

2025-03-21 11:45:01,301 - INFO - Creating simplified test problem
2025-03-21 11:45:01,305 - INFO - Validating optimization inputs...
2025-03-21 11:45:01,307 - INFO - Found 1 hotels in the data
2025-03-21 11:45:01,310 - INFO - Found 2 hawker centers in the data
2025-03-21 11:45:01,311 - INFO - Found 3 attractions in the data
2025-03-21 11:45:01,314 - INFO - Transport matrix contains all required routes (120 total)
2025-03-21 11:45:01,315 - INFO - Budget check passed: 150 >= minimum 70 for hotel and food



===== SIMPLIFIED PROBLEM CHARACTERISTICS =====
Number of locations: 6
  - Hotels: 1
  - Hawkers: 2
  - Attractions: 3
Number of days: 1
Budget: $150
Transport matrix entries: 120
Number of variables: 78
Number of inequality constraints: 99
Number of equality constraints: 14

Time Windows:
  Daily start time: 09:00
  Daily end time: 22:00
  Lunch window: 11:00 - 15:00
  Dinner window: 17:00 - 21:00

===== RUNNING BUILT-IN FEASIBILITY TEST =====


AttributeError: 'TravelItineraryProblem' object has no attribute 'test_feasibility'